# Take a set of calibrations
The script is built for the scriptQueue but for the moment should be run from the notebook

This script is used to take flats with the AuxTel calibration illumination system. <br>
It is being used as a stand-in until the proper functionality is build into the control system.

In [ ]:
import sys
import asyncio
import time
import os
import pickle as pkl
import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

## First, enable LATISS and ATCS and position telescope

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
await latiss.enable()

In [ ]:
await atcs.enable()

In [ ]:
await atcs.prepare_for_flatfield()

In [ ]:
await atcs.disable_dome_following()

In [ ]:
await atcs.slew_dome_to(355.0)

In [ ]:
start_az=181.7
start_el=39.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

## Now connect to the ATCalSys CSCs

In [ ]:
atmonochromator = salobj.Remote(d, 'ATMonochromator')
FiberSpectrograph = salobj.Remote(name="FiberSpectrograph", domain=d, index=3)
electrometer = salobj.Remote(name="Electrometer", domain=d, index=201) 
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=d)

## Bring in needed subroutines

In [ ]:
# functions to run the electromter measurements
def get_file_transfer(lfa):
    filename = os.path.split(lfa.url)[1]
    save_dir = os.path.join(data_dir, filename.split(':')[0])
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_file = os.path.join(save_dir, filename)
    os.system(f"curl {lfa.url} --output {save_file}")
    return save_file

async def elec_meas(exp_time):
    electrometer.evt_largeFileObjectAvailable.flush()
    tmp1=await electrometer.cmd_startScan.set_start(timeout=10)
    await asyncio.sleep(exp_time)
    tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)
    lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    filename = filename.replace(":", "_")
    elec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {elec_filen}")
    hdu = fits.open(f'{elec_filen}')
    data = hdu[1].data
    return filename, lfa, data

async def fiber_meas(exp_time):
    FiberSpectrograph.evt_largeFileObjectAvailable.flush()
    tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=exp_time, numExposures=1)
    lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    filename = filename.replace(":", "_")
    spec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {spec_filen}")
    hdu = fits.open(f'{spec_filen}')
    wavelength = hdu[1].data['wavelength'].flatten()
    spectrum = hdu[0].data
    return filename, lfa, wavelength, spectrum


## Set up electrometer

In [ ]:
await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)  
await electrometer.cmd_setIntegrationTime.set_start(intTime=0.1) 
await electrometer.cmd_setRange.set_start(setRange=-1)

## Take 20 biases for system warm-up

In [ ]:
# Take 20 biases for system warm-up
for i in range(20):
    await latiss.take_bias(1)
    await asyncio.sleep(2.0)


## Now run all three filters

In [ ]:
latiss_grating= 'empty_1'
wave_step = 5 # Wavelength step in nm
exp_time = 60.0
num_latiss_exps = 1

# Dictionary to hold the results
results = {}
for latiss_filter in ['SDSSg_65mm', 'SDSSr_65mm', 'SDSSg_65mm']:
    if latiss_filter =='SDSSg_65mm':
        min_wave = 350; max_wave = 600; grating = 0; slit_1 = 5.0; slit_2 = 5.0
    elif latiss_filter == 'SDSSr_65mm':
        min_wave = 500; max_wave = 750; grating = 1; slit_1 = 5.0; slit_2 = 5.0
    elif latiss_filter == 'SDSSi_65mm':
        min_wave = 640; max_wave = 880; grating = 1; slit_1 = 5.0; slit_2 = 5.0
    else:
        raise "Filter not supported"
    await atmonochromator.cmd_selectGrating.set_start(gratingType=grating, timeout=180)        
    await atmonochromator.cmd_changeSlitWidth.set_start(slit=1, slitWidth=slit_1)  
    await atmonochromator.cmd_changeSlitWidth.set_start(slit=2, slitWidth=slit_2) 
    wave = min_wave
    # This loop steps through the wavelengths for each filter
    while wave < max_wave + wave_step / 2.0:
        await atmonochromator.cmd_changeWavelength.set_start(wavelength=wave)
        key = f"{latiss_filter}_{wave:.1f}"
        filename, electrometer_lfa, electrometer_data = await elec_meas(1.0)
        for (t,current) in electrometer_data:
            electrometer_current = current
        filename, fiber_lfa, wavelengths, spectrum = await fiber_meas(exp_time)
        spec_max = np.max(spectrum)
        # The loop below prevents the spectrum from saturating
        while spec_max > 15900:
            exp_time /= 2.0
            filename, fiber_lfa, wavelengths, spectrum = await fiber_meas(exp_time)
            spec_max = np.max(spectrum)
        await latiss.take_flats(exp_time, num_latiss_exps, filter=latiss_filter, grating=latiss_grating)
        wave += wave_step
        print(electrometer_current, exp_time, spec_max)
        results[key] = {}
        results[key]['electrometer_lfa_url']=electrometer_lfa.url
        results[key]['fiber_lfa_url']=fiber_lfa.url
        results[key]['electrometer_current']=electrometer_current
        results[key]['fiber_data']=[wavelengths, spectrum, exp_time]
        print(f"Done with {latiss_filter}, {wave}")

In [ ]:
# Pickle the data
filename = "/scratch/cslage/filter_cals/filter_cal_02May23.pkl"
file = open(filename, "wb")
pkl.dump(results, file)
file.close()